In [307]:
import os
import polars as pl

In [308]:
pl.Config(tbl_rows=100)

In [309]:
df = pl.read_parquet(os.path.join("data","cnb_ceska_proza.parquet")).rename({'autorstvo_kod':'100_7'})
df = pl.concat([df, pl.read_parquet(os.path.join("data","cnb_ceska_poezie.parquet"))])
spisovatelstvo = list(set(df.select(pl.col('100_7')).drop_nulls().to_series().to_list()))

In [310]:
df = pl.read_parquet(os.path.join("data/cnb_sloupce","100.parquet"))

In [311]:
df = df.filter(pl.col("100_7").is_in(spisovatelstvo))

In [ ]:
aut = pl.read_parquet(os.path.join("data","aut_vyber.parquet")).explode("370_c").filter(pl.col("370_c").str.contains("Česk")).explode("100_7").select(["100_7","374_a","375_a","678_a"])

In [ ]:
aut

In [ ]:
df = df.join(aut, left_on="100_7", right_on="100_7", how="left")

In [ ]:
df

In [ ]:
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","008.parquet")), left_on="001", right_on="001", how="left")

In [ ]:
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","245.parquet")), left_on="001", right_on="001", how="left")

In [ ]:
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","655.parquet")), left_on="001", right_on="001", how="left")

In [ ]:
df

In [ ]:
df.explode("655_a").group_by("655_a").len().top_k(100, by="len")

In [ ]:
df = df.explode("655_a")

In [ ]:
poezie = df.filter(pl.col('655_a').str.contains('poez|poetr')).with_columns(kategorie = pl.lit('poezie'))
proza = df.filter(pl.col('655_a').str.contains('román|fiction|novel|příbě|povíd|stor|próz')).with_columns(kategorie = pl.lit('próza'))
vzpominky = df.filter(pl.col('655_a').str.contains('vzpomínky|autobiographical')).with_columns(kategorie = pl.lit('vzpomínky'))

In [ ]:
do_grafu = pl.concat([poezie, proza, vzpominky]).unique(subset=['100_a','245_a'],keep='first')

In [ ]:
do_grafu

In [ ]:
from src.najdi_rok import najdi_rok

In [ ]:
do_grafu = do_grafu.with_columns(pl.col('008').map_elements(najdi_rok, return_dtype=int).alias('rok'))

In [ ]:
def narozeni(udaje):
    try:
        return int(udaje.split("-")[0])
    except:
        return None
        
def umrti(udaje):
    try:
        return int(udaje.split("-")[1])
    except:
        return None

In [ ]:
do_grafu = do_grafu.with_columns(pl.col("100_d").map_elements(narozeni).alias('narozeni')).with_columns(pl.col("100_d").map_elements(umrti).alias('umrti'))

In [ ]:
do_grafu = do_grafu.drop_nulls("narozeni")

In [ ]:
do_grafu = do_grafu.with_columns((pl.col("rok") - pl.col("narozeni")).alias("vek"))

In [ ]:
do_grafu = do_grafu.filter(pl.col("vek") > 6)

In [ ]:
do_grafu.sort(by="vek").head(n=10)

In [ ]:
do_grafu = do_grafu.filter((pl.col("umrti") >= pl.col('rok')) | pl.col("umrti").is_null())

In [ ]:
do_grafu = do_grafu.filter(pl.col("vek") < 110)

In [ ]:
import altair as alt
alt.data_transformers.disable_max_rows()

In [ ]:
do_grafu

In [ ]:
do_grafu2 = do_grafu.filter(pl.col("rok") < 1900).group_by(["vek","kategorie"]).len().rename({"len":"počet","vek":"věk"})

alt.Chart(do_grafu2.to_pandas()).mark_line().encode(
    alt.X('věk:Q').axis(domain=False, tickSize=0),
    alt.Y('počet:Q').stack('zero').axis(None),
    alt.Color('kategorie:N')
)

In [ ]:
do_grafu2 = do_grafu.filter(pl.col("rok").is_between(1875,1925)).group_by(["vek","kategorie"]).len().rename({"len":"počet","vek":"věk"})

alt.Chart(do_grafu2.to_pandas()).mark_line().encode(
    alt.X('věk:Q').axis(domain=False, tickSize=0),
    alt.Y('počet:Q').stack('zero').axis(None),
    alt.Color('kategorie:N')
)

In [ ]:
do_grafu2 = do_grafu.filter(pl.col("rok").is_between(1925,1975)).group_by(["vek","kategorie"]).len().rename({"len":"počet","vek":"věk"})
print(len(do_grafu.filter(pl.col("rok").is_between(1925,1975))))

alt.Chart(do_grafu2.to_pandas()).mark_line().encode(
    alt.X('věk:Q').axis(domain=False, tickSize=0),
    alt.Y('počet:Q').stack('zero').axis(None),
    alt.Color('kategorie:N')
)

In [ ]:
do_grafu2 = do_grafu.filter(pl.col("rok").is_between(1995,2025)).group_by(["vek","kategorie"]).len().rename({"len":"počet","vek":"věk"})
print(len(do_grafu.filter(pl.col("rok").is_between(1995,2025))))

alt.Chart(do_grafu2.to_pandas()).mark_line().encode(
    alt.X('věk:Q').axis(domain=False, tickSize=0),
    alt.Y('počet:Q').stack('zero').axis(None),
    alt.Color('kategorie:N')
)

In [ ]:
do_grafu2 = do_grafu.filter(pl.col("rok") > 1950).group_by(["vek","kategorie"]).len().rename({"len":"počet","vek":"věk"})

alt.Chart(do_grafu2.to_pandas()).mark_line().encode(
    alt.X('věk:Q').axis(domain=False, tickSize=0),
    alt.Y('počet:Q').stack('zero').axis(None),
    alt.Color('kategorie:N')
)